In [21]:
! pip install transformers 
! pip install datasets 
! pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [24]:
import transformers
print(transformers.__version__)

4.17.0


In [25]:
model_checkpoint = "bert-base-multilingual-cased"
batch_size = 16

In [26]:
!apt install git-lfs
!git config --global user.email "patelkrinal214@gmail.com"
!git config --global user.name "krinal214"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 50 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 1s (2240 kB/s)  33m

78Selecting previously unselected package git-lfs.
(Reading database ... 107701 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
7Progress: [  0%] [..........................

### Loading the dataset

In [27]:
from datasets import load_dataset, load_metric

In [28]:
datasets=load_dataset("tydiqa","secondary_task")
datasets

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 49881
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5077
    })
})

In [29]:
dataset1=load_dataset('krinal214/squad_ben_tel_que_only')

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/krinal214--squad_ben_tel_que_only-730f3cb29b9521a5/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [30]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 139958
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 21106
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 34990
    })
})

In [31]:
dataset2=load_dataset('squad')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [33]:
from datasets import concatenate_datasets
datasets['train']=concatenate_datasets([datasets['train'],dataset1['train']])
datasets['train']=concatenate_datasets([datasets['train'],dataset2['train']])

In [34]:
import re
def filter_dataset(ds,lang):
    def check(d):
        for i in lang:
            if re.search(i,d):
                return True
        return False
    ds = ds.filter(lambda x: check(x['id']))
    return ds

In [35]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [36]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=8):
    picks = random.sample(range(0, len(dataset)-1), num_examples)
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

### Preparing the training and validation dataset

In [37]:
# datasets["train"]=filter_dataset(datasets["validation"],['bengali'])
datasets["validation"]=filter_dataset(datasets["validation"],['bengali', 'telugu'])

In [38]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,56e880ec99e8941900975e46-39845,Bern,"As of 2003[update] the average price to rent an average apartment in Bern was 1108.92 Swiss francs (CHF) per month (US$890, £500, €710 approx. exchange rate from 2003). The average rate for a one-room apartment was 619.82 CHF (US$500, £280, €400), a two-room apartment was about 879.36 CHF (US$700, £400, €560), a three-room apartment was about 1040.54 CHF (US$830, £470, €670) and a six or more room apartment cost an average of 2094.80 CHF (US$1680, £940, €1340). The average apartment price in Bern was 99.4% of the national average of 1116 CHF. The vacancy rate for the municipality, in 2010[update], was 0.45%.",২০১০ সালে বার্নের জন্য শূন্যতার হার কত ছিল?,"{'text': ['.45%'], 'answer_start': [610]}"
1,5733c743d058e614000b622d-8492,University_of_Notre_Dame,"George Gipp was the school's legendary football player during 1916–20. He played semiprofessional baseball and smoked, drank, and gambled when not playing sports. He was also humble, generous to the needy, and a man of integrity. It was in 1928 that famed coach Knute Rockne used his final conversation with the dying Gipp to inspire the Notre Dame team to beat the Army team and ""win one for the Gipper."" The 1940 film, Knute Rockne, All American, starred Pat O'Brien as Knute Rockne and Ronald Reagan as Gipp. Today the team competes in Notre Dame Stadium, an 80,795-seat stadium on campus. The current head coach is Brian Kelly, hired from the University of Cincinnati on December 11, 2009. Kelly's record in midway through his sixth season at Notre Dame is 52–21. In 2012, Kelly's Fighting Irish squad went undefeated and played in the BCS National Championship Game. Kelly succeeded Charlie Weis, who was fired in November 2009 after five seasons. Although Weis led his team to two Bowl Championship Series bowl games, his overall record was 35–27, mediocre by Notre Dame standards, and the 2007 team had the most losses in school history. The football team generates enough revenue to operate independently while $22.1 million is retained from the team's profits for academic use. Forbes named the team as the most valuable in college football, worth a total of $101 million in 2007.",1916 నుండి 1920 వరకు నోట్రే డేమ్లో ఏ ప్రముఖ ఫుట్బాల్ ఆటగాడు ఆడాడు?,"{'text': ['George Gipp'], 'answer_start': [0]}"
2,570b00026b8089140040f69f-35789,Videoconferencing,"This technique was very expensive, though, and could not be used for applications such as telemedicine, distance education, and business meetings. Attempts at using normal telephony networks to transmit slow-scan video, such as the first systems developed by AT&T Corporation, first researched in the 1950s, failed mostly due to the poor picture quality and the lack of efficient video compression techniques. The greater 1 MHz bandwidth and 6 Mbit/s bit rate of the Picturephone in the 1970s also did not achieve commercial success, mostly due to its high cost, but also due to a lack of network effect —with only a few hundred Picturephones in the world, users had extremely few contacts they could actually call to, and interoperability with other videophone systems would not exist for decades.",স্লো-স্ক্যান ভিডিও প্রেরণ করার জন্য প্রথম সিস্টেমটি কে তৈরি করেছিলেন?,"{'text': ['AT&T Corporation'], 'answer_start': [259]}"
3,5726d6265951b619008f7f7e,Chinese_characters,"The earliest confirmed evidence of the Chinese script yet discovered is the body of inscriptions on oracle bones from the late Shang dynasty (c. 1200–1050 BC). These symbols, carved on pieces of bone and turtle shell being sold as ""dragon bones"" for medicinal purposes, were identified as Chinese writing by scholars in 1899. By 1928, the source of the oracle bones had been traced to a village near Anyang in Henan Province, which was excavated by the Academia Sinica between 1928 and 1937. Over 150,000 fragments have been found.",Where were the symbols carved on?,"{'text': ['pieces of bone and turtle shell'], 'answer_start

In [39]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpxmi9514l


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
creating metadata file for /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpvu148fs2


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
creating metadata file for /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidde

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
creating metadata file for /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp3w8m9kfn


Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
creating metadata file for /root/.cache/huggingface/transformers/46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449

In [40]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 160  # The allowed overlap between two part of the context when splitting is performed.

In [41]:
datasets['train']=datasets['train'].filter(lambda example: len(example['question'])<=max_length)

  0%|          | 0/278 [00:00<?, ?ba/s]

In [42]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 277435
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 782
    })
})


In [43]:
def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1 :
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
    
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [44]:
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

  0%|          | 0/278 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [45]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abs

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-multilingual-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
creating metadata file for /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
loading weights file https://huggingface.co/bert-base-multilingual-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/0a3fd51713dcbb4def175c7f85bddc995d5976ce1dde327f99104e4d33069f17.aa7be4c79d76f4066d9b354496ea477c9ee39c5d889156dd1efb680643c2b052
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cl

In [46]:
from transformers import TrainingArguments, Trainer
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-model",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub = True,
    push_to_hub_model_id = "bert-all-squad_que_translated",
    save_steps=50000
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.7/site-packages/transformers/training_args.py:952: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case krinal214/bert-all-squad_que_translated).
  FutureWarning,


In [47]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [48]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/krinal214/bert-all-squad_que_translated into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Download file pytorch_model.bin:   0%|          | 15.6k/676M [00:00<?, ?B/s]

Download file runs/Apr10_10-36-45_18630015cae6/events.out.tfevents.1649587016.18630015cae6.39.0: 100%|########…

...

Clean file runs/Apr13_15-14-43_2fdee3ca9973/1649863055.1230729/events.out.tfevents.1649863055.2fdee3ca9973.74.…

Clean file runs/Apr13_15-14-43_2fdee3ca9973/events.out.tfevents.1649863055.2fdee3ca9973.74.0:  19%|#8        |…

Clean file runs/Apr10_10-36-45_18630015cae6/1649587016.5285215/events.out.tfevents.1649587016.18630015cae6.39.…

Clean file pytorch_model.bin:   0%|          | 1.00k/676M [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [49]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 288175
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 930
    })
})


### Training the model

In [50]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 288175
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 18011


Epoch,Training Loss,Validation Loss
1,1.074600,0.517371


***** Running Evaluation *****
  Num examples = 930
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18011, training_loss=1.3209338833361635, metrics={'train_runtime': 11811.9302, 'train_samples_per_second': 24.397, 'train_steps_per_second': 1.525, 'total_flos': 5.64743946542976e+16, 'train_loss': 1.3209338833361635, 'epoch': 1.0})

In [51]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

### Evaluating the model

In [52]:
max_answer_length = 50
n_best_answers=20

In [53]:
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 50):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[start_index])==0
                        or len(offset_mapping[end_index])==0
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    try:
                        start_char = offset_mapping[start_index][0]
                        end_char = offset_mapping[end_index][1]
                        valid_answers.append(
                            {
                                "score": start_logits[start_index] + end_logits[end_index],
                                "text": context[start_char: end_char]
                            }
                        )
                    except IndexError:
                        continue

        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]
        
    return predictions


In [54]:
import collections
def evaluate(ds,wrong_pred,language):
    val_dataset=filter_dataset(ds,language)
    validation_features = val_dataset.map(
        prepare_validation_features,
        batched=True,
        remove_columns=val_dataset.column_names
    )
    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
    
    examples = val_dataset
    features = validation_features

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    final_predictions = postprocess_qa_predictions(val_dataset, validation_features, raw_predictions.predictions,n_best_answers,max_answer_length)
    metric = load_metric("squad")
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in val_dataset]
    for j in range(len(references)):
        wrong=True
        for t in references[j]["answers"]["text"]:
            if normalize_answer(formatted_predictions[j]['prediction_text'])==normalize_answer(t):
                wrong=False
                break
        if wrong:
            wrong_pred.append({"id":references[j]["id"],"prediction":formatted_predictions[j]["prediction_text"],"original:":t})
    return metric.compute(predictions=formatted_predictions, references=references)

### Result

In [55]:
import string
eval_languages = [['bengali'], ['telugu']]
wrong_prediction={}
wrong_prediction['bengali']=[]
wrong_prediction['telugu']=[]
wrong_prediction['all']=[]
for lang in eval_languages:
    output = evaluate(datasets["validation"],wrong_prediction[lang[0]],lang)
    print(lang[0],':',output)

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 144
  Batch size = 16


Post-processing 113 example predictions split into 144 features.


  0%|          | 0/113 [00:00<?, ?it/s]

bengali : {'exact_match': 69.02654867256638, 'f1': 79.33941181728792}


  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 786
  Batch size = 16


Post-processing 669 example predictions split into 786 features.


  0%|          | 0/669 [00:00<?, ?it/s]

telugu : {'exact_match': 70.70254110612855, 'f1': 83.80529491318201}


In [56]:
print('wrong predictions(bengali)',len(wrong_prediction['bengali']))
print('wrong predictions(telugu)',len(wrong_prediction['telugu']))

wrong predictions(bengali) 35
wrong predictions(telugu) 196


In [57]:
output = evaluate(datasets["validation"],wrong_prediction['all'],['telugu','bengali'])
print('overall',output)

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id. If offset_mapping, example_id are not expected by `BertForQuestionAnswering.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 930
  Batch size = 16


Post-processing 782 example predictions split into 930 features.


  0%|          | 0/782 [00:00<?, ?it/s]

overall {'exact_match': 70.46035805626599, 'f1': 83.15996909497737}


### Visualizing wrong predictions

In [58]:
picks = random.sample(range(0, len(wrong_prediction['bengali'])-1), min(40,len(wrong_prediction['bengali'])-1))
for i in picks:
    print(wrong_prediction['bengali'][i])

{'id': 'bengali--9113522782624640859-1', 'prediction': 'চট্টগ্রাম বিশ্ববিদ্যালয়', 'original:': 'চট্টগ্রাম'}
{'id': 'bengali--2217820890808407965-4', 'prediction': '১২ নভেম্বর', 'original:': '১৯৮৪ সালের ২৪ অক্টোবর'}
{'id': 'bengali-5195671202360353215-59', 'prediction': '১৯০৫ হতে ১৯১১ খ্রীস্টাব্দ', 'original:': '১৯০৫'}
{'id': 'bengali--2660184311490318740-0', 'prediction': 'অভিজিৎ রায় (', 'original:': 'মুক্তমনা'}
{'id': 'bengali--3420112403184584609-0', 'prediction': '১৯৭১ সালে', 'original:': '১৯৭১'}
{'id': 'bengali--394254364202012577-0', 'prediction': 'মাউন্টেইন ভিউ', 'original:': 'মাউন্টেইন ভিউতে'}
{'id': 'bengali--8177010464512382506-0', 'prediction': 'দার্শনিক আল-কিন্দি', 'original:': '৮ম থেকে ১২শ শতাব্দী'}
{'id': 'bengali--5802537117646660783-14', 'prediction': 'মুর্থির', 'original:': 'মুর্থি'}
{'id': 'bengali-3279821707062003108-0', 'prediction': '২০০২ এর ৩১শে মার্চ', 'original:': '২০০২'}
{'id': 'bengali--8404986622637348674-3', 'prediction': 'পাকিস্তান বনাম আইসিসির নতুন সদস্য 

In [59]:
picks = random.sample(range(0, len(wrong_prediction['telugu'])-1), min(40,len(wrong_prediction['telugu'])-1))
for i in picks:
    print(wrong_prediction['telugu'][i])

{'id': 'telugu-1915828106004454396-0', 'prediction': '515 హెక్టార్లలో', 'original:': '515 హెక్టార్ల'}
{'id': 'telugu-4617795002680831857-1', 'prediction': '14వ శతాబ్దం', 'original:': '14వ శతాబ్దంలో'}
{'id': 'telugu--3900695699038118719-0', 'prediction': '1314 హెక్టార్లలో', 'original:': '1314 హెక్టార్ల'}
{'id': 'telugu-5578742953116674238-0', 'prediction': '1289 హెక్టార్లలో', 'original:': '1289 హెక్టార్ల'}
{'id': 'telugu-32885819921748529-1', 'prediction': 'జమ్ము మరియు కాశ్మీర్ లోని ఫగ్వాల్\u200cలో', 'original:': 'జమ్ము మరియు కాశ్మీర్ లోని ఫగ్వాల్\u200c'}
{'id': 'telugu--7150452250756477828-2', 'prediction': '490 హెక్టార్లలో', 'original:': '490 హెక్టార్ల'}
{'id': 'telugu--591970387500414500-0', 'prediction': '91 హెక్టార్లలో', 'original:': '91 హెక్టార్ల'}
{'id': 'telugu--6140267610656239828-1', 'prediction': '15 హెక్టార్లలో', 'original:': '15 హెక్టార్ల'}
{'id': 'telugu--7197225676827031746-0', 'prediction': 'హీరోయిన్', 'original:': 'శాంతి నిలయం'}
{'id': 'telugu--647216164607568860-0', 'p

In [60]:
trainer.push_to_hub()

Saving model checkpoint to bert-base-multilingual-cased-finetuned-model
Configuration saved in bert-base-multilingual-cased-finetuned-model/config.json
Model weights saved in bert-base-multilingual-cased-finetuned-model/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased-finetuned-model/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/676M [00:00<?, ?B/s]

Upload file runs/Apr14_18-46-57_f48159eef302/events.out.tfevents.1649962203.f48159eef302.33.0: 100%|##########…

Upload file runs/Apr14_18-46-57_f48159eef302/1649962203.10821/events.out.tfevents.1649962203.f48159eef302.33.1…

...

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/krinal214/bert-all-squad_que_translated
   5dd1d6b..e45c6ec  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/krinal214/bert-all-squad_que_translated/commit/5dd1d6b71d9d272a9368145468b1374328b1c4bd'